In [1]:
#####################
#### DATAMODULE #####
#####################

import torch
import pytorch_lightning as pl
from torch.utils.data import random_split, DataLoader, Dataset

# Note - you must have torchvision installed for this example
from torchvision import transforms

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
from torchvision.io import read_image
import torchvision
import json
import matplotlib.animation as animation
from matplotlib.patches import Ellipse
# import matplotlib.transforms as transforms
import shutil


## Data loader

class ImageDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        img_dir = os.path.join(data_dir,'images')
        lbl_dir = os.path.join(data_dir,'labels')
        self.img_ID = [os.path.splitext(img)[0] for img in os.listdir(img_dir)]
        self.img_type = [os.path.splitext(img)[1] for img in os.listdir(img_dir)]
        self.img_dir = img_dir
        self.lbl_dir = lbl_dir
        self.transform = lambda x : -1 + 2*(x/torch.max(x))
        #self.transform = torchvision.transforms.Normalize()
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_ID)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, f'{self.img_ID[idx]}{self.img_type[idx]}')
        image = read_image(img_path, torchvision.io.ImageReadMode.UNCHANGED).to(torch.float32)
        
        #lbl_path = open(os.path.join(self.lbl_dir, f'{self.img_ID[idx]}.txt'),'r')
        #lines = lbl_path.readlines()
        
#         labels = torch.tensor([lbl[0] for lbl in lines])
        labels = 0

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            labels = self.target_transform(labels)
        return image, labels
    
class ImageDataModule():
    def __init__(self, data_dir, img_size, batch_size = 8, cuda = False):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.transform = transforms.Compose([transforms.CenterCrop(img_size),transforms.ToTensor()])
        self.target_transform = transforms.Compose([transforms.ToTensor()])
        self.cuda = cuda

    def setup(self, stage: str):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            self.image_train = ImageDataset(self.data_dir + '/train/')
            self.image_val = ImageDataset(self.data_dir + '/valid/')

        if stage == "test":
            self.image_test = ImageDataset(self.data_dir + '/test/')
            
        if stage == "outliers":
            self.image_outliers = ImageDataset(self.data_dir + '/outliers/')
    def train_dataloader(self):
        
        if self.cuda:
            sampler = DistributedSampler(self.image_train)
        else:
            sampler = None
            
        return DataLoader(self.image_train,
                          batch_size=self.batch_size,
                          pin_memory=True,
                          shuffle=False,
                          sampler = sampler)
         
    def valid_dataloader(self):
        
        
        if self.cuda:
            sampler = DistributedSampler(self.image_val)
        else:
            sampler = None
            
        return DataLoader(self.image_val,
                          batch_size=self.batch_size,
                          pin_memory=True,
                          shuffle=False,
                          sampler=sampler)
    

    def test_dataloader(self):
        return DataLoader(self.image_test, batch_size=self.batch_size)
    
    def outliers_dataloader(self):
        return DataLoader(self.image_outliers, batch_size=self.batch_size)
    
    
## TEST 

data_dir = './data/mars_rover'
datamodule = ImageDataModule(data_dir)

datamodule.setup('fit')

train_data = datamodule.train_dataloader()

# plt.imshow(torch.permute(train_data.dataset[0][0],(1,2,0)))
# for i,l in train_data:
#     print(i.shape)

In [2]:
import torch.nn as nn

class Encoder(nn.Module):

    def __init__(self, encoded_space_dim=2, fc2_input_dim=128, num_channels=[8, 16, 32], in_chan = 3):
        super().__init__()
        self.encoded_space_dim = encoded_space_dim
        self.num_channels = num_channels

        ### Convolutional section
        self.compress_layer = nn.Conv2d(3, 3, 3, stride=2, padding=0)

        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(3, self.num_channels[0], 3, stride=2, padding=0),
            nn.BatchNorm2d(self.num_channels[0]),
            nn.ReLU(),
            nn.Conv2d(self.num_channels[0], self.num_channels[1], 3, stride=2, padding=0),
            nn.BatchNorm2d(self.num_channels[1]),
            nn.ReLU(),
            nn.Conv2d(self.num_channels[1], self.num_channels[2], 3, stride=2, padding=0),
            nn.ReLU()
        )

        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        ### Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(9 * 9 * self.num_channels[2], fc2_input_dim),
            nn.ReLU(),
            nn.Linear(fc2_input_dim, encoded_space_dim * 2)
        )

    def forward(self, x):
        for _ in range(3):
            x = self.compress_layer(x)
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        x = self.encoder_lin(x)
        mu, logsigmasq = x[:, :self.encoded_space_dim], x[:, self.encoded_space_dim:]
        return mu, logsigmasq


class Decoder(nn.Module):

    def __init__(self, encoded_space_dim=2, fc2_input_dim=128, num_channels=[32, 16, 8], in_chan = 3):
        super().__init__()
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, fc2_input_dim),
            nn.ReLU(),
            nn.Linear(fc2_input_dim, 9 * 9 * num_channels[0]),
            nn.ReLU()
        )

        self.unflatten = nn.Unflatten(dim=1,
                                      unflattened_size=(num_channels[0], 9, 9))

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(num_channels[0], num_channels[1], 3,
                               stride=2, output_padding=(0, 0)),
            nn.BatchNorm2d(num_channels[1]),
            nn.ReLU(),
            nn.ConvTranspose2d(num_channels[1], num_channels[2], 3, stride=2,
                               padding=0, output_padding=(0, 0)),
            nn.BatchNorm2d(num_channels[2]),
            nn.ReLU(),
            nn.ConvTranspose2d(num_channels[2], 2*in_chan, 3, stride=2,
                               padding=0, output_padding=(0, 0))
        )


        self.decompress_layer_0 = nn.ConvTranspose2d(2*in_chan, 2*in_chan, 3, stride=2, padding=0, output_padding=(0,0))
        self.decompress_layer_1 = nn.ConvTranspose2d(2*in_chan, 2*in_chan, 3, stride=2, padding=0, output_padding=(1,1))

    def forward(self, x):

        x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
#         x = self.decompress_layer_0(x)
        for _ in range(2):
            x = self.decompress_layer_0(x)
        x = self.decompress_layer_1(x)
        mu, logsigmasq = x[:, :3, :, :], x[:, 3:, :, :]
        return mu, logsigmasq

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import os


r = 202211010
np.random.seed(r)
torch.manual_seed(r)

D = 1 # number of modalities
dim = 64 # dimension of each modality (assume to be same)
K = 2
Z = 4
latent_dim = Z

lr = 1e-3
num_epochs = 1500
batch_size = 8
num_workers = 8
em_reg = 1e-6
logsigmasq_reg = em_reg

means_hist = []
mu_c_hist = []
logsigmasq_c_hist = []
gamma_c_train_hist = []
gamma_c_val_hist = []


w_rec = 1.0
w_reg = 1.0
w_entr = 1.0


sim_name = F'gmvae_ld{latent_dim}_nc{K}_rec{w_rec}_reg{w_reg}_entr{w_entr}'
data_dir = './data/mars_rover'

my_datamodule = ImageDataModule(data_dir)
my_datamodule.setup('fit')

save_every = 1

# N_train, W_img, H_img = train_dataset.data.shape  # 60000, 28, 28
# N_test, _, _ = test_dataset.data.shape  # 10000, 28, 28

train_loader = my_datamodule.train_dataloader()
valid_loader = my_datamodule.valid_dataloader()

train_labels = torch.Tensor([int(train_loader.dataset[i][1]) for i in range(len(train_loader.dataset))])
valid_labels = torch.Tensor([int(valid_loader.dataset[i][1]) for i in range(len(valid_loader.dataset))])

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
# from kmeans_pytorch import kmeans

import matplotlib.pyplot as plt
import numpy as np

def encoder_step(x, encoder, decoder):
    """ Computes a stochastic estimate of the rescaled evidence lower bound

    Args:
        x_list: length-D list of (N, data_dim) torch.tensor
        encoder_list: length-D list of Encoder
        decoder_list: length-D list of Decoder
        params: dictionary of other parameters
    Returns:
        elbo: a (,) torch.tensor containing the estimate of the ELBO
    """
    mu, logsigmasq = encoder.forward(x)
    return mu, logsigmasq + logsigmasq_reg

def em_step(z, mu, params, update_by_batch=False):

    mu_c = params['mu_c'].to(device)  # (K, Z)
    logsigmasq_c = params['logsigmasq_c'].to(device)  # (K, Z)
    sigma_c = torch.exp(0.5 * logsigmasq_c)
    pi_c = params['pi_c'].to(device)

    log_prob_zc = Normal(mu_c, sigma_c).log_prob(z.unsqueeze(dim=1)).sum(dim=2) + torch.log(pi_c)  #[N, K]
    log_prob_zc -= log_prob_zc.logsumexp(dim=1, keepdims=True)
    gamma_c = torch.exp(log_prob_zc) + em_reg

    denominator = torch.sum(gamma_c, dim=0).unsqueeze(1)
    mu_c = torch.einsum('nc,nz->cz', gamma_c, mu) / denominator
    logsigmasq_c = torch.log(torch.einsum('nc,ncz->cz', gamma_c, (mu.unsqueeze(dim=1) - mu_c) ** 2)) - torch.log(denominator)

    if not update_by_batch:
        return gamma_c, mu_c, logsigmasq_c

    else:
        hist_weights = params['hist_weights'].to(device)
        hist_mu_c = params['hist_mu_c'].to(device)
        hist_logsigmasq_c = params['hist_logsigmasq_c'].to(device)

        curr_weights = denominator
        new_weights = hist_weights + curr_weights
        new_mu_c = (hist_weights * hist_mu_c + curr_weights * mu_c) / new_weights
        new_logsigmasq_c = torch.log(hist_weights * torch.exp(hist_logsigmasq_c) + curr_weights * torch.exp(logsigmasq_c)) - torch.log(new_weights)
        # new_logsigmasq_c = torch.log(torch.exp(torch.log(hist_weights) + hist_logsigmasq_c) +
        #                              torch.exp(torch.log(curr_weights) + logsigmasq_c)) - torch.log(new_weights)

        params['hist_weights'] = new_weights
        params['hist_mu_c'] = new_mu_c
        params['hist_logsigmasq_c'] = new_logsigmasq_c
        return gamma_c, new_mu_c, new_logsigmasq_c


def decoder_step(x, z, encoder, decoder, params, mu, logsigmasq, gamma_c):
    """
    Computes a stochastic estimate of the ELBO.
    :param x_list: length-D list of (N, data_dim) torch.tensor
    :param z: MC samples of the encoded distributions
    :param encoder_list: length-D list of Encoder
    :param decoder_list: length-D list of Decoder
    :param params: dictionary of non-DNN parameters
    :return:
        elbo: (,) tensor containing the elbo estimation
    """
    sigma = torch.exp(0.5 * logsigmasq)
    mu_c = params['mu_c']
    logsigmasq_c = params['logsigmasq_c']
    pi_c = params['pi_c']

    elbo = 0
    
    reconstruction = 0
    regularization = 0
    entropy = 0
    
    mu_, logsigmasq_ = decoder.forward(z)
    reconstruction += Normal(mu_, torch.exp(0.5 * logsigmasq_)).log_prob(x).sum()
        
    regularization = - 0.5 * torch.sum(gamma_c * (logsigmasq_c + (sigma.unsqueeze(1) ** 2 + (mu.unsqueeze(1) - mu_c) ** 2) /
                                         torch.exp(logsigmasq_c)).sum(dim=2))
    
    entropy = torch.sum(gamma_c * (torch.log(pi_c) - torch.log(gamma_c))) + 0.5 * torch.sum(1 + logsigmasq)

    elbo = w_rec*reconstruction + w_reg*regularization + w_entr*entropy
    
    return elbo, reconstruction, regularization, entropy

In [ ]:
# initialize latent GMM model parameters
params = {}
device = 0

pi_variables = torch.zeros(K, requires_grad = True, device = device)
params['pi_c'] = torch.ones(K) / K
torch.manual_seed(r)
params['mu_c'] = torch.rand((K, Z)) * 2.0 - 1.0
params['mu_c'] = params['mu_c']
params['logsigmasq_c'] = torch.zeros((K, Z))

# initialize neural networks
encoder_list = []
decoder_list = []
trainable_parameters = []
trainable_parameters.append(pi_variables)

torch.manual_seed(r)
encoder = Encoder(encoded_space_dim = latent_dim).to(device)
decoder = Decoder(encoded_space_dim = latent_dim).to(device)

trainable_parameters += list(encoder.parameters()) + list(decoder.parameters())

optimizer = optim.Adam(trainable_parameters, lr=lr)


# training
train_loss = torch.zeros(num_epochs)
rec_loss = torch.zeros(num_epochs)
reg_loss = torch.zeros(num_epochs)
entr_loss = torch.zeros(num_epochs)

valid_loss = torch.zeros(num_epochs)
pi_history = torch.zeros((num_epochs, K))
min_valid_loss = torch.inf
epoch_list = []
train_loss_list = []
valid_loss_list = []

rec_loss_list = []
reg_loss_list = []
entr_loss_list = []

for epoch in range(num_epochs):
    encoder.train()
    decoder.train()

    train_elbo = 0
    rec_elbo = 0
    reg_elbo = 0
    entr_elbo = 0
    gamma_c_epoch = []
    params['hist_weights'] = torch.zeros((K, 1)).clone().detach()
    params['hist_mu_c'] = torch.zeros((K, Z)).clone().detach()
    params['hist_logsigmasq_c'] = torch.zeros((K, Z)).clone().detach()

    for (batch_idx, batch_x) in enumerate(train_loader):
        
        x = batch_x[0].to(device)
        
        optimizer.zero_grad()
        pi_c = torch.exp(pi_variables) / torch.sum(torch.exp(pi_variables))
#         print(f'pi variables: {pi_variables}')
#         print(f'pi_c: {pi_c}')

        params['pi_c'] = pi_c

        mu, logsigmasq = encoder_step(x, encoder, decoder)
#         print(f'mu: {mu}')
#         print(f'logsigmasq: {logsigmasq}')
        sigma = torch.exp(0.5 * logsigmasq)
        torch.manual_seed(r)
        eps = Normal(0, 1).sample(mu.shape).to(device)
        z = mu + eps * sigma
#         print(f'z: {z}')

        with torch.no_grad():
            gamma_c, mu_c, logsigmasq_c = em_step(z, mu, params, update_by_batch=True)
            
#         print(f'gamma_c: {gamma_c}')
#         print(f'mu_c: {mu_c}')
#         print(f'logsigmasq_c: {logsigmasq_c}')
        params['mu_c'] = mu_c
        params['logsigmasq_c'] = logsigmasq_c
        gamma_c_epoch.append(gamma_c)

        elbo, rec, reg, entr = decoder_step(x, z, encoder, decoder, params, mu, logsigmasq, gamma_c)
        
        train_elbo += elbo.item()
        rec_elbo += rec.item()
        reg_elbo += reg.item()
        entr_elbo += entr.item()
        loss = - elbo
        
#         print(f'step: {batch_idx} | train_loss: {-train_elbo}')
        loss.backward()
        optimizer.step()
        
    gamma_c_train_hist.append(torch.vstack(gamma_c_epoch))

    encoder.eval()
    decoder.eval()

    valid_elbo = 0
    gamma_c_epoch = []
    with torch.no_grad():
        for (batch_idx, batch_x) in enumerate(valid_loader):
#             print('VALIDATION')
            x = batch_x[0].to(device)
            mu, logsigmasq = encoder_step(x, encoder, decoder)
#             print(f'mu: {mu}')
#             print(f'logsigmasq: {logsigmasq}')
            sigma = torch.exp(0.5 * logsigmasq)
            torch.manual_seed(r)
            eps = Normal(0, 1).sample(mu.shape).to(device)
            z = mu + eps * sigma
#             print(f'z: {z}')
            with torch.no_grad():
                gamma_c, _, _ = em_step(z, mu, params)
#             print(f'gamma_c: {gamma_c}')
            gamma_c_epoch.append(gamma_c)
            elbo, rec, reg, entr  = decoder_step(x, z, encoder, decoder, params, mu, logsigmasq, gamma_c)
            valid_elbo += elbo.item()
            
    gamma_c_val_hist.append(torch.vstack(gamma_c_epoch))

    train_elbo /= len(train_loader.dataset)
    valid_elbo /= len(valid_loader.dataset)
    # print('====> Epoch: {} Train ELBO: {:.4f} '.format(epoch, train_elbo))
    print('====> Epoch: {} Train ELBO: {:.4f} Val ELBO: {:.4f}'.format(epoch, train_elbo, valid_elbo))

    train_loss[epoch] = - train_elbo
    rec_loss[epoch] = - rec_elbo
    reg_loss[epoch] = - reg_elbo
    entr_loss[epoch] = - entr_elbo
    valid_loss[epoch] = - valid_elbo
    pi_history[epoch] = params['pi_c']

    if epoch % save_every == 0:
        epoch_list.append(epoch)
        train_loss_list.append(train_loss[epoch].item())
        valid_loss_list.append(valid_loss[epoch].item())
        rec_loss_list.append(rec_loss[epoch].item())
        reg_loss_list.append(reg_loss[epoch].item())
        entr_loss_list.append(entr_loss[epoch].item())
        # Plot the first two dimensions of the latents
        with torch.no_grad():
            means = []
            # labels = []
            for batch_x in train_loader:
                x = batch_x[0].to(device)
                mean, _ = encoder_step(x, encoder, decoder)
                means.append(mean)
                
                # labels.append(batch_label)

        means = torch.vstack(means).cpu()
        # labels = torch.hstack(labels)
        means_hist.append(means)
        mu_c_hist.append(params['mu_c'].cpu())
        logsigmasq_c_hist.append(params['logsigmasq_c'].cpu())
        fig, ax = plt.subplots(figsize=(6, 5))
        for i in range(K):
            means_i = means[train_labels == i]
            ax.scatter(means_i[:, 0], means_i[:, 1], alpha=0.25, label=str(i))
        for i in range(K):
            ax.plot(params['mu_c'].cpu()[i, 0], params['mu_c'].cpu()[i, 1], 'x', markersize=12) #, label='$\mu$' + str(i + 1))
        
        ax.set_xlabel('$z_1$')
        ax.set_ylabel('$z_2$')
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.tight_layout()
        plt.close()



====> Epoch: 0 Train ELBO: -1181861.5126 Val ELBO: -1005192.7614
====> Epoch: 1 Train ELBO: -634175.4725 Val ELBO: -105426.8876
====> Epoch: 2 Train ELBO: -25561.3768 Val ELBO: 84014.1024
====> Epoch: 3 Train ELBO: 131526.6517 Val ELBO: 241228.0934
====> Epoch: 4 Train ELBO: 187156.7730 Val ELBO: 288028.7244
====> Epoch: 5 Train ELBO: 205100.3277 Val ELBO: -42959.6819
====> Epoch: 6 Train ELBO: 208730.5198 Val ELBO: 219280.6204
====> Epoch: 7 Train ELBO: 222541.6588 Val ELBO: 351017.7997
====> Epoch: 8 Train ELBO: 240508.8355 Val ELBO: 350586.7670
====> Epoch: 9 Train ELBO: 254153.9512 Val ELBO: 342937.8622
====> Epoch: 10 Train ELBO: 265093.9903 Val ELBO: 357411.9673
====> Epoch: 11 Train ELBO: 266537.5737 Val ELBO: 334134.4659
====> Epoch: 12 Train ELBO: 268428.7328 Val ELBO: 309306.0909
====> Epoch: 13 Train ELBO: 280760.4525 Val ELBO: 354236.9716
====> Epoch: 14 Train ELBO: 291501.8950 Val ELBO: 384733.8310
====> Epoch: 15 Train ELBO: 303206.3602 Val ELBO: 374949.2912
====> Epoch: 

====> Epoch: 132 Train ELBO: 777512.2431 Val ELBO: 512100.7585
====> Epoch: 133 Train ELBO: 747579.0925 Val ELBO: 570009.2585
====> Epoch: 134 Train ELBO: 777518.2923 Val ELBO: 200162.6119
====> Epoch: 135 Train ELBO: 701038.1370 Val ELBO: 633717.4915
====> Epoch: 136 Train ELBO: 808043.5183 Val ELBO: 501792.4432
====> Epoch: 137 Train ELBO: 782350.2229 Val ELBO: 629133.1676
====> Epoch: 138 Train ELBO: 793050.2872 Val ELBO: -92575.3058
====> Epoch: 139 Train ELBO: 712933.2260 Val ELBO: 607975.3722
====> Epoch: 140 Train ELBO: 787024.8176 Val ELBO: 526641.3097
====> Epoch: 141 Train ELBO: 787452.7942 Val ELBO: 592942.3125
====> Epoch: 142 Train ELBO: 795463.8378 Val ELBO: 634951.5994
====> Epoch: 143 Train ELBO: 750170.0095 Val ELBO: 465170.5227
====> Epoch: 144 Train ELBO: 742987.6768 Val ELBO: 503914.3097
====> Epoch: 145 Train ELBO: 758882.0063 Val ELBO: 534015.9375
====> Epoch: 146 Train ELBO: 809380.1187 Val ELBO: 603033.1477
====> Epoch: 147 Train ELBO: 805895.0101 Val ELBO: 5441

====> Epoch: 263 Train ELBO: 878185.3403 Val ELBO: 461287.8920
====> Epoch: 264 Train ELBO: 896579.9527 Val ELBO: 438005.3011
====> Epoch: 265 Train ELBO: 907009.6641 Val ELBO: 463540.2926
====> Epoch: 266 Train ELBO: 916825.2140 Val ELBO: 505571.1619
====> Epoch: 267 Train ELBO: 908904.1900 Val ELBO: 559741.5511
====> Epoch: 268 Train ELBO: 889131.3371 Val ELBO: 475276.5085
====> Epoch: 269 Train ELBO: 890631.0694 Val ELBO: 556902.5710
====> Epoch: 270 Train ELBO: 897002.7910 Val ELBO: 382791.6009
====> Epoch: 271 Train ELBO: 879157.3125 Val ELBO: 515920.8537
====> Epoch: 272 Train ELBO: 880970.0878 Val ELBO: 581939.3381
====> Epoch: 273 Train ELBO: 902009.8889 Val ELBO: 520836.5824
====> Epoch: 274 Train ELBO: 929652.0732 Val ELBO: 536743.1989
====> Epoch: 275 Train ELBO: 921620.7563 Val ELBO: 439405.8260
====> Epoch: 276 Train ELBO: 906564.1970 Val ELBO: 491599.1037
====> Epoch: 277 Train ELBO: 865494.8428 Val ELBO: 599035.1562
====> Epoch: 278 Train ELBO: 892081.9097 Val ELBO: 5694

In [ ]:
path = f'./results/{sim_name}/'
#plt.plot(train_e)
if os.path.exists(path):
    shutil.rmtree(path)
os.makedirs(path)

# Compute the mean of the latents given the data
encoder.eval()

with torch.no_grad():
    means = []
    # labels = []
    for batch_x in train_loader:
        x = batch_x[0].to(device)
        mean, _ = encoder_step(x, encoder, decoder)
        means.append(mean)
        # labels.append(batch_label)

means = torch.vstack(means)
# labels = torch.hstack(labels)

with torch.no_grad():
    gamma_c, mu_c, logsigmasq_c = em_step(means, means, params, update_by_batch=False)

my_datamodule.setup('test')
test_loader = my_datamodule.test_dataloader()
with torch.no_grad():
    means_test = []
    for batch_x in test_loader:
        x = batch_x[0].to(device)
        mean, _ = encoder_step(x, encoder, decoder)
        means_test.append(mean)
        # labels.append(batch_label)
means_test = torch.cat(means_test)

my_datamodule.setup('outliers')
outliers_loader = my_datamodule.outliers_dataloader()
with torch.no_grad():
    means_outliers = []
    for batch_x in outliers_loader:
        x = batch_x[0].to(device)
        mean, _ = encoder_step(x, encoder, decoder)
        means_outliers.append(mean)
        # labels.append(batch_label)
means_outliers = torch.cat(means_outliers)

# Plot the first two dimensions of the latents
# plot_params()
fig, ax = plt.subplots(figsize=(6, 5))
cluster_means = torch.zeros((K, Z))
for i in range(K):
    means_i = means[train_labels == i].cpu()
    ax.scatter(means_i[:, 0], means_i[:, 1], alpha=0.25, label=str(i))
    cluster_means[i] = torch.mean(means_i, dim=0)
for i in range(K):
    ax.plot(params['mu_c'].cpu()[i, 0], params['mu_c'].cpu()[i, 1], 'x', markersize=12, color = 'k')  # , label='$\mu$' + str(i + 1))

ax.scatter(means_test[:, 0].cpu(), means_test[:, 1].cpu(), alpha=1, color = 'g')
ax.scatter(means_outliers[:, 0].cpu(), means_outliers[:, 1].cpu(), alpha=1, color = 'r')
ax.set_xlabel('$z_1$ mean')
ax.set_ylabel('$z_2$ mean')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.savefig(path +'latent.png', dpi=600)


# Plot training loss vs. epoch number
plt.figure(figsize=(4.5, 4))
const = min(train_loss)
train_loss_adjusted = train_loss - const + 10.
# val_loss_adjusted = val_loss - const + 10.
plt.semilogy(train_loss_adjusted, label='train')
# plt.semilogy(val_loss_adjusted, label='val')
plt.xlabel("number of epochs")
# plt.legend()
plt.tight_layout()
plt.savefig(path +'train_loss.png', dpi=600)
plt.close()

# Plot the training and validation loss vs. epoch number
plt.figure(figsize=(4.5, 4))
const = min(min(train_loss), min(valid_loss))
train_loss_adjusted = train_loss - const + 10.
valid_loss_adjusted = valid_loss - const + 10.
plt.semilogy(train_loss_adjusted, label='train')
plt.semilogy(valid_loss_adjusted, label='val')
plt.xlabel("number of epochs")
plt.legend()
plt.tight_layout()
plt.savefig(path +'loss.png', dpi=600)
plt.close()

# Plot the history of pi
plt.figure(figsize=(4.5, 4))
for i in range(K):
    plt.plot(pi_history[:, i].detach().numpy(), label='$\pi$' + str(i+1))
plt.xlabel("number of epochs")
plt.legend()
plt.tight_layout()
plt.close()

In [ ]:
# plt.plot(train_loss_list)
# plt.plot(val_loss_list)
plt.plot(rec_loss_list)
plt.plot(reg_loss_list)
plt.plot(entr_loss_list)


In [ ]:
results_dic = {}
results_dic['epochs'] = epoch_list
results_dic['train_loss_epoch'] = train_loss_list
results_dic['valid_loss_epoch'] = valid_loss_list
results_dic['reconstruction_loss_epoch'] = rec_loss_list
results_dic['regularization_loss_epoch'] = reg_loss_list
results_dic['entropy_loss_epoch'] = entr_loss_list
results_dic['mu_c'] = [mu_c.cpu().numpy().tolist() for mu in mu_c_hist]
results_dic['logsigmasq_c'] = [logsig.cpu().numpy().tolist() for logsig in logsigmasq_c_hist]
results_dic['means'] = [mu.cpu().numpy().tolist() for mu in means_hist]
results_dic['pi'] = [pi.cpu().detach().numpy().tolist() for pi in pi_history]
results_dic['gamma_c_train'] = [gamma.cpu().detach().numpy().tolist() for gamma in gamma_c_train_hist]
results_dic['train_labels'] = train_labels.tolist()
results_dic['gamma_c_val'] = [gamma.cpu().detach().numpy().tolist() for gamma in gamma_c_val_hist]
results_dic['val_labels'] = valid_labels.tolist()

with open(path + f'GMVAE_rocks_K1_Z4.json', 'w') as outfile:
    json.dump(results_dic, outfile)
    
%xdel results_dic

In [ ]:
xmax = np.max(means_hist[0].numpy()[:,0])
ymax = np.max(means_hist[0].numpy()[:,1])
xmin = np.min(means_hist[0].numpy()[:,0])
ymin = np.min(means_hist[0].numpy()[:,1])
max_epoch = np.max(np.array(epoch_list))

for i in range(len(means_hist)):
    means = means_hist[i].numpy()
    if np.max(means[:,0]) > xmax:
        xmax = np.max(means[:,0])
    if np.max(means[:,1]) > ymax:
        ymax = np.max(means[:,1])
    if np.min(means[:,0]) < xmin:
        xmin = np.min(means[:,0])
    if np.min(means[:,1]) < ymin:
        ymin = np.min(means[:,1])
    

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=20, metadata=dict(artist='Me'))

fig, axs = plt.subplots(1,2,figsize=(13, 5))
train_loss_list = np.array(train_loss_list)
valid_loss_list = np.array(valid_loss_list)
rec_loss_list = np.array(rec_loss_list)
reg_loss_list = np.array(reg_loss_list)
entr_loss_list = np.array(entr_loss_list)
epoch_list = np.array(epoch_list)

fig.suptitle(f'GMVAE | latent dim. = {latent_dim}, # clusters = {K}', fontsize=16)
def animate(j):
    print(j)
    #Axis 1
    means = means_hist[j]
    mu_c = mu_c_hist[j]
    sigmaq_c = np.exp(logsigmasq_c_hist[j].numpy())
    axs[0].clear()
    axs[1].clear()
    for i in range(2):
        means_i = means[train_labels == i]
        axs[0].scatter(means_i[:, 0], means_i[:, 1], alpha=0.25, label=str(i))
    for i in range(K):
        axs[0].plot(mu_c[i, 0], mu_c[i, 1], 'x', markersize=12) #, label='$\mu$' + str(i + 1))
        c = axs[0].get_lines()[0].get_color()
        ellipse = Ellipse((mu_c[i, 0], mu_c[i, 1]),
        width=2*3*sigmaq_c[i, 0],
        height=2*3*sigmaq_c[i, 1],
        facecolor='None',
        edgecolor = c,
        linestyle = '--')
        axs[0].add_patch(ellipse)
        
    axs[0].set_xlabel('$z_1$')
    axs[0].set_ylabel('$z_2$')
    axs[0].set_xlim([xmin, xmax])
    axs[0].set_ylim([ymin, ymax])
    axs[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))

    #Axis 2
    axs[1].plot(epoch_list[0:j], train_loss_list[0:j] / train_loss[0],'k')
    axs[1].plot(epoch_list[0:j], valid_loss_list[0:j] / valid_loss[0],'m')
    axs[1].plot(epoch_list[0:j], rec_loss_list[0:j] / rec_loss[0],'r--')
    axs[1].plot(epoch_list[0:j], reg_loss_list[0:j] / reg_loss[0],'g--')
    axs[1].plot(epoch_list[0:j], entr_loss_list[0:j] / entr_loss[0],'b--')
    axs[1].set_xlabel('$epoch #$')
    axs[1].set_ylabel('$loss$')
    axs[1].set_xlim([0, max_epoch])
#     ax2.set_ylim([ymin, ymax])
    axs[1].legend(['train loss', 'val loss', 'rec. loss', 'reg. loss', 'entropy loss'], loc='center left', bbox_to_anchor=(1, 0.5))
    fig.tight_layout()

In [ ]:
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(epoch_list), repeat=False)
ani.save(path + 'training_video.mp4', writer=writer, dpi=600)

In [ ]:
path